# K-En Yakın Komşu (KNN)

* Tahminler gözlem benzerliğine göre yapılır.
* Öklid ya da benzeri bir uzaklık hesabı ile her bir gözleme uzaklık hesaplanır.
* En yakın K adet gözlemin y değerlerinin ortalaması alınır.
* y tahmini değer bulunur.

## KNN Basamakları

* Komşu sayısını belirle (K)
* Bilinmeyen nokta ile diğer tüm noktalar ile arasındaki uzaklıkları hesapla
* Uzaklıkları sırala ve belirlenen k sayısına göre en yakın olan k gözlemi seç
* Sınıflandırma ise en sık sınıf, regresyon ise ortalama değeri tahmin değeri olarak ver.

parametrik olmayan bir öğrenme türüdür. büyük veri setlerinde performans olarak çok iyi olduğu söylenemez ama sınıflandırma problemleri için ortaya çıkmış daha sonra regresyon modellerine uyarlanmıştır. kullanılması uygulanması kolay olduğundan tercih edilir.

Tahminler gözlem benzerliğine göre yapılır.
gözlem birimi(bağımsız değişkenler), bağlımlı değişken değerim nedir? diye sorar. KNN'de sana en benzer olan arkadaşlarına bakacağım, der ve bu gözlem birimine en yakın olan k (3, 5...) tane gözlem birimini inceleyip kendisine en yakın olanlara göre bu gözlem biriminin bağımlı değişkenini bulmuş oluyor.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

In [16]:
# uyarıları kapatmak için

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
from sklearn.neighbors import KNeighborsRegressor
# knn modelini oluşturdum
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [4]:
# ön tanımlı olarak komşuluk parametresi (k) değeri 5 dir.
# model hiperparametresi olarak anlandırabileceğimiz knn'in parametresi komşuluk sayısıdır.
knn_model.n_neighbors

5

In [ ]:
# knn, farklı k değerlerine göre farklı değerler verebilir.
# optimize etmemiz gereklidir.

# K-En Yakın Komşu (KNN) - Tahmin

In [5]:
# test setine göre y tahmin değerleri
y_pred = knn_model.predict(X_test)
y_pred[:10]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ,  325.5   ,
        216.5   ,  101.5   ,  982.    ,  886.6666])

In [6]:
# rmse test hatası
np.sqrt(mean_squared_error(y_test, y_pred))

426.6570764525201

In [7]:
RMSE = [] 

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse) 
    print("k =" , k , "için RMSE değeri: ", rmse)

k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  179.52761335480352
k = 3 için RMSE değeri:  205.20157172291863
k = 4 için RMSE değeri:  220.5139794876305
k = 5 için RMSE değeri:  239.6467132541376
k = 6 için RMSE değeri:  243.5904190007242
k = 7 için RMSE değeri:  258.1478781634636
k = 8 için RMSE değeri:  266.05374203349805
k = 9 için RMSE değeri:  269.73782093553376
k = 10 için RMSE değeri:  271.2798300436963


In [ ]:
# k = 2 için RMSE değeri:  179.52761335480352

Cross Validation yöntemiyle farklı k değerlerine karşılık olacak şekilde nasıl bir seçim işlemi yapacağız

# K-En Yakın Komşu (KNN) - Model Tuning

GridSearchCV metoduyla optimum k sayısını belirleme işlemini gerçekleştireceğiz.

GridSearchCV yöntemiyle hiperparametre araması yapacağız. GridSearchCV, bir ızgara mantığıyla bir olası parametre setinin verilip tüm olası kombinasyonların denenmesi anlamına gelir. 

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
knn_params = {"n_neighbors": np.arange(1,30,1)}
# 1den 30a kadar olan komşuluk (k) değerlerini modelde deneyip hangisinin daha iyi olduğunu bulacağız

In [12]:
np.arange(1,30,1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [13]:
# knn model nesnesi
knn = KNeighborsRegressor()

In [14]:
# knn cross validation nesnesi
knn_cv_model = GridSearchCV(knn, knn_params, cv=10)

In [15]:
# modeli eğittim
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

model kurma işlemini gerçekleştirdim. cv yöntemine göre bize uygun olabilecek parametreleri aslında şu anda hepsini denedi ve bunun neticesinde uygun olan parametreyi bulmuş oldu.

In [17]:
# bu en uygun parametreyi bulmak için knn_cv_model nesnesinin içerisinden
# 8'i en uygun parametre değeri olarak bulmuş oldu
knn_cv_model.best_params_["n_neighbors"]

8

In [18]:
# train seti için validate edilmiş ve edilmemiş rmse hatalarını gözlemleyelim
RMSE = []
RMSE_CV = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train)
    rmse = np.sqrt(mean_squared_error(y_train, y_pred))
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model,
                                         X_train,
                                         y_train,
                                         cv=10,
                                         scoring="neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k =", k, "için RMSE değeri:", rmse, "RMSE_CV değeri: ", rmse_cv)

k = 1 için RMSE değeri: 0.0 RMSE_CV değeri:  325.3947514706382
k = 2 için RMSE değeri: 179.52761335480352 RMSE_CV değeri:  293.24000183333817
k = 3 için RMSE değeri: 205.20157172291863 RMSE_CV değeri:  283.7486667487823
k = 4 için RMSE değeri: 220.5139794876305 RMSE_CV değeri:  286.3240222024089
k = 5 için RMSE değeri: 239.6467132541376 RMSE_CV değeri:  290.0705466132226
k = 6 için RMSE değeri: 243.5904190007242 RMSE_CV değeri:  298.1263115575851
k = 7 için RMSE değeri: 258.1478781634636 RMSE_CV değeri:  294.77070479194987
k = 8 için RMSE değeri: 266.05374203349805 RMSE_CV değeri:  291.98672028891235
k = 9 için RMSE değeri: 269.73782093553376 RMSE_CV değeri:  295.7162739573105
k = 10 için RMSE değeri: 271.2798300436963 RMSE_CV değeri:  301.31047022701154


hata kareler ortalamasının karekökü değerleri (RMSE değeri), bunlar test ve train üzerinden direk yapılan hesaplamalardır.

validate edilmiş model üzerinden elde edilmiş hatalar (RMSE_CV değeri)

best_params fonksiyonu RMSE değerine bakarak bir sırlama yapmadığı için böyle bir durumla karşılaşılıyor.  best_params fonksiyonu "mean_validation_score" değerine göre bir sıralama yapıyor. Bu nedenle RMSE değerlerine bakıldığında farklı bir sonuç çıkıyor.

In [17]:
# bu en uygun parametreyi bulmak için knn_cv_model nesnesinin içerisinden
# 8'i en uygun parametre değeri olarak bulmuş oldu
knn_cv_model.best_params_["n_neighbors"]

8

In [19]:
# final modelimiz
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [20]:
knn_tuned.fit(X_train, y_train) # modeli eğittik

KNeighborsRegressor(n_neighbors=8)

In [21]:
# test seti için y tahmini değerleri
y_pred = knn_tuned.predict(X_test)

In [22]:
# rmse test hata
np.sqrt(mean_squared_error(y_test, y_pred))

413.7094731463598